# Project 2: A minimal model training experiment

**Goal**:

- Create a [PyTorch LightningModule](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html) named `ImageClassifier` that holds a convolutional network with ResNet18 backbone.
- Learn to prepare the dataset and dataloader to be compatible with the model.
- Understand a minimal set of components of a model training pipeline: loss, optimizer, metrics, training loop.
- Assemble the components to train a model using the dataset class and the dataloader created in the previous object.
- Learn how to visualize model predictions.
- Understand the concept of fine-tuning and the benefits of starting from a pre-trained model. (may move to the next project)
- Understand the benefits and options of a dataloader. (may move to the next project)

**Acceptance Criteria**:

- Implement a test that checks a simple ImageClassifier can be predict on an image that has the correct shape.
- The `ImageClassifier` can be trained on the CIFAR10 dataset, showing decreasing loss and accuracy for several epochs.

**Resources**:

- If you want to use docker to run code in this notebook, `pytorch/pytorch:1.12.0-cuda11.3-cudnn8-runtime` is a good choice.

## Step 1: Create a model using `LightningModule`

`LightningModule` is a convenient and structured way to implement a PyTorch model, as well as its training and validation behaviors. For more information, please refer to the [documentation](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html) for `LightningModule`.

### Installation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Install the dependencies:
!pip install torch torchvision pytorch-lightning==1.6.4 torchmetrics mlflow
# If the above installation fails, try:
# !pip install torch==1.12.0 torchvision==0.13.0 pytorch-lightning==1.6.4 torchmetrics mlflow

### A simple image classifier module

In [ ]:
import pytorch_lightning as pl
from torchvision.models import resnet18

class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = resnet18(num_classes=10, weights=None)
    
    def forward(self, x):
        return self.net(x)

In [ ]:
model = ImageClassifier()

### Testing the forward path of the model

As a trainable function, the model is callable. The model's forward path (as defined in `forward()`) is the normal execution of the function. We can test this by passing an image to the model.

In [ ]:
assert callable(model), "The model is not a callable object."
print("Model is callable.")

#### What is a valid input?

To test the forward path of the model, we need to pass a valid input. The input should be a tensor of shape `(b, 3, 32, 32)` where `b` is the batch size (an integer).

**Your Task**: Fix the code below to pass the test.

**Tips:** `torch.from_numpy()` can be used to convert a numpy array to a tensor.


In [ ]:
import numpy as np
import torch

# TODO: The test is broken. Please fix it.
def test_model_can_predict_on_a_random_image():
    # input_image = np.ones(shape=(3, 224, 224), dtype=np.float32)
    input_image = torch.from_numpy(np.ones(shape=(1, 3, 224, 224), dtype=np.float32))
    output = model(input_image)  # run the model on the input image
    assert output.shape == (1, 10)  # is the output shape correct?


test_model_can_predict_on_a_random_image()
print("Test passed")

## Step 2: Prepare the dataset and data loader

You have two choices here and either way would work.

1. Use the `CIFAR10` dataset class provided by `torchvision`. This is easier.
   ```
   from torchvision.datasets import CIFAR10

   cifar10_train = CIFAR10(train=True, download=True, root="./cifar10")
   cifar10_val = CIFAR10(train=False, download=True, root="./cifar10")
   ```
2. Or, reuse the `CustomDataset` from the previous object.

In [ ]:
# TODO: use `torchvision.datasets.CIFAR10` or copy over the code from the previous project.
# TODO: inspect the dataset by print out one example from it.
# TODO: Make sure the image is a Tensor or np.array (so that the data loader can turn it into
#   a Tensor-typed image batch), rather than a PIL image.
from torchvision.datasets import CIFAR10
from torchvision import transforms


cifar10_train = CIFAR10(...)
cifar10_val = CIFAR10(...)

To make the dataset compatible with PyTorch models, if you use `torchvision.datasets.CIFAR10`, the image needs to a `Tensor` rather than a PIL image. Please add `transform=transforms.ToTensor` when calling the class to creating the dataset object. More information about transforms can be found [here](https://pytorch.org/vision/stable/transforms.html).

## Step 3: Training the model

#### Loss

A loss is a function that takes the model's output (also called predictions) and the ground truth (also called "targets") as input and returns a scalar value. The loss is used as the feedback mechantism to optimize the model.

For classification, the loss is typically the cross-entropy. Use `torch.nn.functional.cross_entropy` when the model output are logits or `torch.nn.functional.nll_loss` when the outputs are probabilities (a.k.a. softmax). Loss functions can have requirements on the type (`dtype`) of input. `cross_entropy`, for example, expects targets to be "Long" or `np.int64`.

**Your Task**: The following code almost works but there is a bug related to data type. Please fix it.

In [ ]:
from torch.nn import functional as F

# To simplify the code, assume there are 3 image classes.
y_pred = torch.from_numpy(np.array([[0, 0, 5]], dtype=np.float32))
y_true = torch.from_numpy(np.array([1], dtype=np.float32))
print(F.cross_entropy(y_pred, y_true))

**Your Task**: Can you make the loss lower? Try changing `y_pred` in the next cell.

In [ ]:
# TODO: change y_pred to make the loss lower.
y_pred = torch.from_numpy(np.array([[0, 0, 5]], dtype=np.float32))
y_true = torch.from_numpy(np.array([1], dtype=int))
print(F.cross_entropy(y_pred, y_true))

#### Optimizer

The gradient over model parameters (a.k.a. model weights) is computed of the loss function, and informs how the model parameters should be updated. The specifics of how the model is updated is handled by an optimizer.

An optimizer can be created by passing the model parameters (`model.parameters()`) to the optimizer constructor, as well as learning rate, momentum, and other hyper-parameters. Below is a minimal example of using an optimizer.

```python
for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
```

When using `LightningModule`, this is taken care of by `LightniningModule` under the hood so you don't have to write the boilerplate code as below (pseudo-code):

```
# put model in train mode and enable gradient calculation
model.train()
torch.set_grad_enabled(True)

outs = []
for batch_idx, batch in enumerate(train_dataloader):
    loss = training_step(batch, batch_idx)
    outs.append(loss.detach())

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()
```

For more details, please refer to [the documentation for PyTorch optimizers](https://pytorch.org/docs/stable/optim.html) and [the documentation for `LightningModule`](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html).

#### Training Step

We can start specifying the training behavior to the model by adding the `training_step` method in the `ImageClassifier` class.

In [ ]:
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = resnet18(num_classes=10, weights=None)
        self.automatic_optimization = False  # 
    
    def forward(self, x):
        return self.net(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    
    def loss(self, y_hat, y):
        return F.cross_entropy(y_hat, y)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        
        return {"loss": loss}

**Your task**: Implement the following unit test. Use the data loader to get a batch of data.
Then, call the `training_step` and assert that the return value is a dictionary that looks
like this: `{'loss': 0.5}`. 

In [ ]:
from torch.utils.data import DataLoader


def test_training_step_works():
    model = ImageClassifier()
    # TODO: Use the data loader to get a batch of data.
    #    Then, call the `training_step` and assert that the return value is a dictionary that looks
    #    like this: {'loss': 0.5}.
    # ...
    

test_training_step_works()

### Metrics

A metric is a function that takes the model's output and the ground truth as input and returns a scalar value. The metric is used to evaluate the model's performance. It is mainly used to monitor the training progress and does not directly influence the model's training behavior. However, it can influence model selection and early stopping.

For classification, the metric is typically the accuracy. We will use `torchmetrics.Accuracy` in this project.

In [ ]:
from torchmetrics import Accuracy

# Again, for simplicity, assume there are only 3 imge classes.
y_pred = torch.from_numpy(np.array([[0, 0, 5], [2, -1, 1]], dtype=np.float32))
# The following line is optional and shouldn't change the result.
# y_pred = F.softmax(y_pred, dim=1)
y_true = torch.from_numpy(np.array([1, 0], dtype=int))
acc = Accuracy(num_classes=3)
acc.update(y_pred, y_true)
print("Accuracy:", acc.compute())

### Training loop

A training loop manages the training process that iteratively passes batches of training examples to the model and running the training step. 

The training steps are organized into "epochs", where an epoch can be a single pass through the entire training dataset, or it can be simply a predefined number of training steps. At the end of an epoch, the model is used predict on the validation dataset and calculate accuracy metrics.

To do this, we need to add `validation_step` and `validation_epoch_end` methods and accuracy metrics to the model.

In [ ]:
from torchmetrics import Accuracy


class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = resnet18(num_classes=10, weights=None)
        self.val_accuracy = Accuracy(num_classes=10)
    
    def forward(self, x):
        return self.net(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0001)
    
    def loss(self, y_hat, y):
        return F.cross_entropy(y_hat, y)
    
    def training_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)

        # Note: The return value can be None, a loss tensor, or a dictionary with a "loss" key.
        return {"loss": loss}
    
    def validation_step(self, batch, batch_idx=None):
        x, y = batch
        y_hat = self(x)
        self.val_accuracy.update(preds=y_hat, target=y)
        val_loss = self.loss(y_hat, y)
        return {"loss": val_loss}

### Start model training

**Your Task**: In the next cells, fill out the details in the training loop implementation with the help of the following pseudo-code examples.

The training loop can be described on a high level:

```python
for epoch in epochs:
    for train_step in train_steps_per_epoch:
        # TODO: loop over the training dataset and run training steps
        pass
    
    # TODO: loop over the validation dataset and run validation steps.
```

Pseudo code for fetching the next batch of data for training or validation:

```python
batch = next(my_data_iterator)
```

Pseudo code for back-propagation to update the model weights:

```python
optimizer.zero_grad()
your_training_loss.backward()
optimizer.step()
```

In [ ]:
from tqdm import tqdm

epochs = 6
train_steps_per_epoch = 100
val_steps_per_epoch = 10

train_dataloader = DataLoader(cifar10_train, batch_size=16)
val_dataloader = DataLoader(cifar10_val, batch_size=16)


def get_infinite_data_iterator(dataloader):
  while True:
    for batch in iter(dataloader):
      yield batch


#
# The training loop:
#
def training_loop(model, train_dataloader, val_dataloader):
    train_data_iterator = get_infinite_data_iterator(train_dataloader)
    optimizer = model.configure_optimizers()
    for epoch in range(epochs):
        train_losses = []
        train_steps = tqdm(list(range(train_steps_per_epoch)))
        for train_step in train_steps:
            # TODO: loop over the training dataset and run the training step.
            
            # TODO: Back-prop and update model's weights.
            
            # TODO: append the training loss value to the list `train_losses`.

            avg_train_loss = np.mean(train_losses)
            train_steps.set_postfix({"loss": avg_train_loss})
        print(f"epoch {epoch}: train loss = {avg_train_loss:.5f}")
        
        # TODO: loop over the validation dataset and run validation steps.
        # Just using the first few examples from the validation set.
        val_data_iterator = iter(val_dataloader)
        val_losses = []
        for val_step in range(val_steps_per_epoch):
          # TODO: loop over the validation dataset and run the validation step.
          
          # TODO: append the validation loss value to the list `val_losses`.
        avg_val_loss = np.mean(val_losses)
        val_accuracy = model.val_accuracy.compute()
        print(f"epoch {epoch}: val loss = {avg_val_loss:.5f}, accuracy = {val_accuracy:.3f}")


model_v1 = ImageClassifier()
training_loop(model_v1, train_dataloader, val_dataloader)

## Step 4: Visualize model predictions

Having loss going down and accuracy going up is nice. But to have the peace of mind that the model is working and is indeed better, it is helpful to visualize the model's predictions.

**Your Task**: Complete the following cells to visualize the model's predictions.

First, visualize predictions from a randomly initialized model:

In [ ]:
from matplotlib import pyplot as plt


def show_images(images, labels: list = None, ncols: int = 8):
    nrows = (len(images) + ncols - 1) // ncols
    fig = plt.figure(figsize=(2 * ncols, 2 * nrows), linewidth=10, edgecolor="#04253a")
    for i in range(len(images)):
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(images[i])
        if labels is not None:
          plt.title(labels[i])
        plt.axis('off')
    # fig.set_facecolor("#e1ddbf")
    return fig


def predict_and_show(model, output_path=None):
    batch = next(iter(val_dataloader))
    images, labels = batch
    softmax = model(images)
    softmax = softmax.detach().numpy()
    predicted_labels = np.argmax(softmax, axis=-1)
    CLASSES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog",
              "horse", "ship", "truck"]
    predicted_label_texts = [f"{CLASSES[pred]}({CLASSES[correct]})" for pred, correct in zip(predicted_labels, labels)]
    fig = show_images(images.numpy().transpose(0, 2, 3, 1), predicted_label_texts)
    if output_path:
        fig.savefig(output_path)


# TODO: use an un-trained model to predict on a few images from the validation set.
#   Use the skill learned in project 1 to display a batch of images.
randomly_initialized_model = ImageClassifier()


Next, take the trained model and visualize its predictions on some image examples:

In [ ]:
# TODO: use the trained model (model_v1) to predict on a few images from the validation set.
#   Do predictions look better than the untrained, randomly initialized model?

## Step 5: Using the `Trainer` provided by Pytorch-Lightning

For commonly used computer vision models, it is often enough to the `Trainer` provided by Pytorch-Lightning to handle the training loop. This way, you can run model training with just a couple lines of code.

### Callbacks

For finer controller of the training loop behaviors, you can create a `Callback` subclass, and fill out `on_train_batch_end`, `on_validation_batch_end`, `on_validation_epoch_end` etc.


In [ ]:
from pytorch_lightning.callbacks import Callback


class ImageClassifierCallback(Callback):

    def __init__(self, save_prediction_examples=False, *args, **kwargs):
      super().__init__(*args, **kwargs)
      self.current_epoch = 0
      self.save_prediction_examples = save_prediction_examples
    
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        super().on_train_batch_end(trainer, pl_module, outputs, batch, batch_idx)
        loss = outputs["loss"]
        self.log(name="loss", value=loss.item(), on_step=True)
    
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        super().on_validation_batch_end(trainer, pl_module, outputs, batch, batch_idx, dataloader_idx)
        loss = outputs["loss"]
        self.log(name="loss", value=loss.item(), on_step=True)
        self.log(name="acc", value=pl_module.val_accuracy.compute(), on_step=True)
        return {"loss": loss}
    
    def on_validation_epoch_end(self, trainer, pl_module, *args, **kwargs):
        super().on_validation_epoch_end(trainer, pl_module, *args, **kwargs)
        avg_acc = pl_module.val_accuracy.compute()
        self.log('val_acc', avg_acc, prog_bar=True)
        print("val_accuracy:", avg_acc)
        if self.save_prediction_examples:
            pass  # not implemented
        self.current_epoch += 1

In [ ]:
from pytorch_lightning import Trainer, seed_everything

# Optional: freeze the random seed.
seed_everything(42, workers=True)
trainer = Trainer(deterministic=True, limit_train_batches=100, limit_val_batches=10, max_epochs=5, callbacks=[ImageClassifierCallback()])
model_v2 = ImageClassifier()
trainer.fit(model_v2, train_dataloader, val_dataloader)

In [ ]:
predict_and_show(model_v2)

## Step 6 (optional): Track the model's progress using an experiment tracking tool

Tracking model's quality during training is made easy by great experiment tracking tools like `mlflow`. With a small change to your training code, we will walk through how to use it to track your model's progress in learning.

**Your task**: modify the `ImageClassifierCallback` class above, and make use of the `log_artifact` method of `MLFlowLogger` to log the figure produced by `predict_and_show` at the end of each epoch.

Pseudo code example:
```
experiment_tracker = trainer.logger.experiment
experiment_tracker.log_artifact(run_id=..., local_path=...)
```

In [ ]:
from pytorch_lightning.loggers import MLFlowLogger

trainer = Trainer(
    limit_train_batches=100,
    limit_val_batches=10,
    max_epochs=5,
    logger=MLFlowLogger(experiment_name="image classifier", tracking_uri="./mlruns"),
    callbacks=[ImageClassifierCallback(save_prediction_examples=True)]
)

model_v3 = ImageClassifier()
trainer.fit(model_v3, train_dataloader, val_dataloader)

The tracking information is already saved by MLFlow. If you are running this on a local computer, you can type: 

```mlflow ui```

and view the MLFlow GUI in your browser.

If you are running this in colab or on a remote server, you can use the following code to visualize metrics and artifacts within this notebook.

In [ ]:
import mlflow

client = mlflow.client.MlflowClient(tracking_uri="./mlruns")
exp = client.get_experiment_by_name("image classifier")
runs = client.list_run_infos(exp.experiment_id, order_by=["tag.start_time DESC"])
latest_run = runs[0]
latest_run

In [ ]:
val_acc_history = client.get_metric_history(latest_run.run_id, key="val_acc")
val_acc_history

In [ ]:
import seaborn

steps = [m.step for m in val_acc_history]
val_acc = [m.value for m in val_acc_history]

fig = seaborn.lineplot(steps, val_acc).set(title="val accuracy", xlabel="step", ylabel="value")

The image artifacts that contain prediction examples can also be retrieved and downloaded. Here we don't visualize them again because the images probably already show up in the cell above.

In [ ]:
client.list_artifacts(run_id=latest_run.run_id)

You have completed the project!